# Cài đặt thư viện


- Cài đặt thư viện Selenium, Chromium
- Thiết lập cấu hình Ubuntu cho Chromium

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.0Idxak6oaZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.5DhDBQFsOY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HxjJwTlQrS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

# Thu thập dữ liệu thời tiết

Thiết lập Webdriver cho Chrome

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import re

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

Thu thập dữ liệu thành phố HCM từ 1/1/2021 -> 31/5/2023

In [3]:
from urllib.parse import urlparse
from selenium.common.exceptions import NoSuchElementException

list_web = [
  {'name':'HoChiMinh','url':'https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/{}-{}'},
]

# Tạo DataFrame trống
all_data = pd.DataFrame()

for i in list_web:
  city_name = i['name']
  url_crawl = i['url']

  for year in range(2021, 2024):
    start_month = 1
    end_month = 12

    if year == 2021:
      start_month = 1
    elif year == 2023:
      end_month = 5

    for month in range(start_month, end_month + 1):
      url = url_crawl.format(year, month)

      # Truy cập trang web và cào dữ liệu
      driver.get(url)
      print(f"Đang thu thập dữ liệu từ trang web: {url}")

      # Chờ đến khi có dữ liệu table
      while True:
        try:
          table = driver.find_element(By.XPATH, "//table[@aria-labelledby='History days']")
          break
        except NoSuchElementException:
          time.sleep(1)

      # Chờ đến khi có dữ liệu column
      while True:
        try:
          columns = table.find_elements(By.XPATH, ".//table[@aria-labelledby='Days data']")
          if columns:
              break
        except NoSuchElementException:
          time.sleep(1)

      # Lấy giá trị thời gian và địa điểm
      url_parts = url.split('/')
      date_value = url_parts[-1]

      time_col = columns[0]
      time_rows = time_col.find_elements(By.XPATH, ".//td")
      time_values = [f"{date_value}-{time.text}" for time in time_rows][1:]

      def dailyweather(column):
        position = [1, 2, 0]
        values = []
        for i in position:
          cells = column.find_elements(By.XPATH, f".//td[position() mod 3 = {i}]")
          values.append([cell.text for cell in cells][1:])
        return values

      # Lấy dữ liệu nhiệt độ
      temperature = columns[1]
      t_max_values, t_avg_values, t_min_values = dailyweather(temperature)

      # Điểm sương
      dewpoint = columns[2]
      d_max_values, d_avg_values, d_min_values = dailyweather(dewpoint)

      # Độ ẩm
      humidity = columns[3]
      h_max_values, h_avg_values, h_min_values = dailyweather(humidity)

      # Tốc độ gió
      windspeed = columns[4]
      w_max_values, w_avg_values, w_min_values = dailyweather(windspeed)

      # Tạo DataFrame từ dữ liệu thu thập được trong tháng
      month_data = pd.DataFrame({
        'Time': time_values,
        'Location': city_name,
        'MaxTemperature': t_max_values,
        'AvgTemperature': t_avg_values,
        'MinTemperature': t_min_values,
        'MaxDewPoint': d_max_values,
        'AvgDewPoint': d_avg_values,
        'MinDewPoint': d_min_values,
        'MaxHumidity': h_max_values,
        'AvgHumidity': h_avg_values,
        'MinHumidity': h_min_values,
        'MaxWindspeed': w_max_values,
        'AvgWindspeed': w_avg_values,
        'MinWindspeed': w_min_values
      })

      # Gộp dữ liệu của tháng hiện tại vào DataFrame tổng
      all_data = pd.concat([all_data, month_data], ignore_index=True)

# Ghi DataFrame vào tệp CSV
all_data.to_csv('data.csv', index=False)

print("Dữ liệu đã được ghi vào tệp data.csv thành công")

Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-1
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-2
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-3
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-4
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-5
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-6
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-7
Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2021-8
Đang thu thập dữ liệu từ trang web: https://www.wundergr